<a href="https://colab.research.google.com/github/Kunal02880/Qunatum-phenomeno-/blob/main/Electromagnetically_Induced_Transparency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qutip
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from qutip import *

In [ ]:

def Hamil(Delta01, Delta12, Omega01, Omega12):
    #--------------------------
    # Detunings
    H0_mat = [[Delta01, 0, 0],[0, 0, 0],[0,0,Delta12]]
    H0 = Qobj(np.array(H0_mat))
    #--------------------------
    # Interaction laser 0 <-> 1
    Hint01_mat = [[0, 0.5*Omega01, 0],[0.5*Omega01, 0, 0],[0,0,0]]
    Hint01 = Qobj(np.array(Hint01_mat))
    #--------------------------
    # Interaction laser 1 <-> 2
    Hint12_mat = [[0, 0, 0],[0, 0, 0.5*Omega12],[0,0.5*Omega12,0]]
    Hint12 = Qobj(np.array(Hint12_mat))
    # return the full Hamiltonian of the system
    return H0+Hint01+Hint12

Delta01 = 1
Delta12 = 2
Omega01 = 3
Omega12 = 4
print(Hamil(Delta01, Delta12, Omega01, Omega12))




In [ ]:
def SE_collapse(Gamma10, Gamma12):
    c10_mat = [[0, Gamma10, 0],[0, 0, 0],[0,0,0]]
    c10 = Qobj(np.array(c10_mat))
    c12_mat = [[0, 0, 0],[0, 0, 0],[0,Gamma12,0]]
    c12 = Qobj(np.array(c12_mat))
    return [c10, c12]

Gamma10 = 1.2
Gamma12 = 2

[c10, c12] = SE_collapse(Gamma10, Gamma12)

print(c10)
print(c12)


In [ ]:
# Initial condition
psi0_vet = np.array([1,0,0])
psi0_shanku = Qobj(psi0_vet)
psi0 = fock(3,0)
rho0 = fock_dm(3,0)
rho0_shanku = psi0_shanku*psi0_shanku.dag()

print(rho0)
print(psi0)
print(psi0_shanku)
print(rho0_shanku)

# Projector
rho1 = fock_dm(3,1)
print(rho1)

In [ ]:

Gamma10 = 0.1
Gamma12 = 0.2
Delta01 = 0
Delta12 = 0.1
Omega01 = 0.1
Omega12 = 0.05
dimt = 100
t = np.linspace(0,500,dimt)
#results = mesolve(Hamil(Delta01, Delta12, Omega01, Omega12), \ #Hamiltonian
#                  psi0, \# Initial condition
#                  t, \ #times of evolution
#                  SE_collapse(Gamma10, Gamma12), \ # collapse operators
#                  e_ops=rho1) # projection

results = mesolve(Hamil(Delta01, Delta12, Omega01, Omega12), \
                  psi0, \
                  t, \
                  SE_collapse(Gamma10, Gamma12), \
                  e_ops=rho1)


fig, ax = plt.subplots()
ax.plot(t,results.expect[0]);
ax.set_xlabel('time');
ax.set_ylabel('expectation value');
ax.legend(("rho1"))

In [ ]:
Gamma10 = 0.5
Gamma12 = 0.3

Omega01 = 0.002
Omega12 = 0.05
dimt = 100
t = np.linspace(0,500,dimt)

Delta12 = 0.1

dim_D01 = 201
Delta01 = np.linspace(-1,1,dim_D01)
rho11_last_12on = 0*Delta01
rho11_last_12off = 0*Delta01
for n in range(dim_D01):
    results = mesolve(Hamil(Delta01[n], Delta12, Omega01, Omega12), \
                  psi0, \
                  t, \
                  SE_collapse(Gamma10, Gamma12), \
                  e_ops=rho1)
    rho11_last_12on[n] = results.expect[0][-1]
    resultsoff = mesolve(Hamil(Delta01[n], 0, Omega01, 0), \
                  psi0, \
                  t, \
                  SE_collapse(Gamma10, Gamma12), \
                  e_ops=rho1)
    rho11_last_12off[n] = resultsoff.expect[0][-1]

fig, ax = plt.subplots()
ax.plot(Delta01,rho11_last_12on/np.max(rho11_last_12on));
ax.plot(Delta01,rho11_last_12off/np.max(rho11_last_12off));
ax.set_xlabel('Detuning 01');
ax.set_ylabel('expectation value');
ax.legend(("12 on", "12_off"))

In [ ]:
Gamma10 = 0.5
Gamma12 = 0.3

Omega01 = 0.002
Omega12 = 0.05
dimt = 100
t = np.linspace(0,500,dimt)

Delta12 = 0.1

dim_D01 = 301
Delta01 = np.linspace(-1,1,dim_D01)
dim_D12 = 5
Delta12 = np.linspace(-0.25,0.25,dim_D12)
rho11_last_12on = 0*Delta01
rho11_last_12off = 0*Delta01

labels = []
fig, ax = plt.subplots()
for n12 in range(dim_D12):
    for n in range(dim_D01):
        results = mesolve(Hamil(Delta01[n], Delta12[n12], Omega01, Omega12), \
                      psi0, \
                      t, \
                      SE_collapse(Gamma10, Gamma12), \
                      e_ops=rho1)
        rho11_last_12on[n] = results.expect[0][-1]

    ax.plot(Delta01,rho11_last_12on/np.max(rho11_last_12on));
    labels.append("Delta12 = "+str(Delta12[n12]))


ax.set_xlabel('Detuning 01');
ax.set_ylabel('expectation value');
ax.legend(labels)